### CS Interview QA Chatbot Usage
This workbook contains the code for training and running the a LSTM-RNN model that answers question about CS interview questions.

To run the saved model:
1. [Import libraries](#scrollTo=aO1Gqpuvtmud&line=1&uniqifier=1)
1. [Load the preprocessed dataset](#scrollTo=G3ow6JgothNy)
1. [Run the utility functions](#scrollTo=-NASuplC3w-N&line=4&uniqifier=1)
1. [Define the model architecture](#scrollTo=JTZIHoO-41rf&line=11&uniqifier=1)
1. [Load saved model](#scrollTo=3HQZcsVhKU8I&line=1&uniqifier=1)
1. [Run evaluation functions](#scrollTo=03W05HyA5Sxh&line=1&uniqifier=1)
1. [Evaluate manually (qualitative)](#scrollTo=H_GOPvu_YYwr)
    
    Sample questions from the dataset
        1. what is recursion
        2. how do you explain bestfirst search
        3. how do you explain a thread
        4. how does divide and conquer work
1. [Evaluate with BERT Score (quantitative)](#scrollTo=n2WXvkcxkj6z)


To train from scratch:
1. [Import libraries](#scrollTo=aO1Gqpuvtmud&line=1&uniqifier=1)
1. [Load the preprocessed dataset](#scrollTo=G3ow6JgothNy)
1. [Run the utility functions](#scrollTo=-NASuplC3w-N&line=4&uniqifier=1)
1. [Define the model architecture](#scrollTo=JTZIHoO-41rf&line=11&uniqifier=1)
1. [Run the training functions](#scrollTo=SFjeoyzLfRmz)
1. [Initialise model](#scrollTo=3HQZcsVhKU8I&line=1&uniqifier=1)    
    Set the `loadFilename` variable to `None`
1. [Run evaluation functions](#scrollTo=03W05HyA5Sxh&line=1&uniqifier=1)
1. [Evaluate manually (qualitative)](#scrollTo=H_GOPvu_YYwr)
1. [Evaluate with BERT Score (quantitative)](#scrollTo=n2WXvkcxkj6z)

### Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
FOLDER_PATH = '/content/drive/My Drive/Colab Notebooks/nlc/project'

Mounted at /content/drive


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import pandas as pd


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")

### Loading Saved Data

In [3]:
import pandas as pd

# read from csv
df = pd.read_csv(os.path.join(FOLDER_PATH, 'combined_data.csv'))
df

,Question,Answer
0,how does randomised algorithm work,the algorithm typically uses uniformly random ...
1,what do you mean by bestfirst search,bestfirst search is a search algorithm which e...
2,how do you explain a daemon,daemon disk and execution monitor is a process...
3,what is phonetic algorithm,a phonetic algorithm is an algorithm for index...
4,what do you mean by uniform costsearch,a tree search that finds the lowestcost route ...
...,...,...
3770,explain biasvariance tradeoff,biasvariance tradeoff is a concept in machine ...
3771,what is stochastic gradient descent sgd in mac...,stochastic gradient descent sgd is an optimiza...
3772,explain stochastic gradient descent,stochastic gradient descent sgd is an optimiza...
3773,what is the backpropagation algorithm in machi...,the backpropagation algorithm is a widely used...


In [4]:
# convert the df to 2d list
qa_pairs = df.values.tolist()
len(qa_pairs), qa_pairs[0]

(3775,
 ['how does randomised algorithm work',
  'the algorithm typically uses uniformly random bits as an auxiliary input to guide its behavior in the hope of achieving good performance in the average case over all possible choices of random bits'])

### Utility Functions
- Create vocabulary
- Normalising string
- Prepare data in batches

In [5]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token
UNK_token = 3  # Unknown word token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token: "UNK"}
        self.num_words = 4  # Count SOS, EOS, PAD, UNK

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS", UNK_token: "UNK"}
        self.num_words = 4 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [6]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p, max_length):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < max_length and len(p[1].split(' ')) < max_length

# Filter pairs using filterPair condition
def filterPairs(pairs, max_length):
    return [pair for pair in pairs if filterPair(pair, max_length)]


In [7]:
voc = Voc("qa")
MAX_LENGTH = 256  # Maximum sentence length to consider

# filter pairs based on MAX_LENGTH
print(f'before filtering, no. of qa pairs: {len(qa_pairs)}')
qa_pairs = filterPairs(qa_pairs, MAX_LENGTH)
print(f'after filtering, no. of qa pairs: {len(qa_pairs)}')

before filtering, no. of qa pairs: 3775
after filtering, no. of qa pairs: 3775


In [8]:
# normalise each qa pair and add word from question and answer into vocab
for i in range(len(qa_pairs)):
    question, answer = qa_pairs[i]
    question = normalizeString(question)
    answer = normalizeString(answer)
    voc.addSentence(question)
    voc.addSentence(answer)
    qa_pairs[i] = [question, answer]

vocab_size = voc.num_words # words + 3 tokens (PAD, SOS, EOS)
questions = [pair[0] for pair in qa_pairs]
answers = [pair[1] for pair in qa_pairs]

# preview first 5 questions and answers
qa_pairs[:5]

[['how does randomised algorithm work',
  'the algorithm typically uses uniformly random bits as an auxiliary input to guide its behavior in the hope of achieving good performance in the average case over all possible choices of random bits'],
 ['what do you mean by bestfirst search',
  'bestfirst search is a search algorithm which explores a graph by expanding the most promising node chosen according to a specified rule'],
 ['how do you explain a daemon',
  'daemon disk and execution monitor is a process that runs in the background without users interaction they usually start at the booting time and terminate when the system is shut down'],
 ['what is phonetic algorithm',
  'a phonetic algorithm is an algorithm for indexing of words by their pronunciation'],
 ['what do you mean by uniform costsearch',
  'a tree search that finds the lowestcost route where costs vary']]

In [9]:
import random
import itertools

def indexesFromSentence(voc, sentence):
    """Convert a sentence into a list of word indices, handling unknown words."""
    return [voc.word2index.get(word, UNK_token) for word in sentence.split(' ')] + [EOS_token]

def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    # print('indexes_batch', indexes_batch) # indexes_batch [[34, 35, 36, 37, 38, 39, 40, 2], [3, 4, 5, 6, 7, 2]]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    # print('inputpadList', padList) # padList [(34, 3), (35, 4), (36, 5), (37, 6), (38, 7), (39, 2), (40, 0), (2, 0)]
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    # print('outputpadList',padList) # padList [(1669, 949), (35, 36), (26, 2), (2, 0)]
    mask = binaryMatrix(padList) # 1 for non-pad tokens, 0 for pad tokens
    # print('mask', mask) # mask [[1, 1], [1, 1], [1, 1], [1, 0]]
    mask = torch.ByteTensor(mask)
    # print('mask', mask) # mask [[1, 1], [1, 1], [1, 1], [1, 0]]
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    # print('input_batch', input_batch) # input_batch ['what do you mean by bestfirst search', 'how does randomised algorithm work']
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# # Example for validation
# testing_pairs = [['how are you doing', 'i do good'], ['you are good', 'like you']]
# small_batch_size = 2
# # batches = batch2TrainData(voc, [random.choice(qa_pairs) for _ in range(small_batch_size)])
# # batches = batch2TrainData(voc, qa_pairs[:small_batch_size])
# batches = batch2TrainData(voc, testing_pairs)
# input_variable, lengths, target_variable, mask, max_target_len = batches

# print("input_variable:", input_variable) # shape (max_input_sequence_length, batch_size)
# print("lengths:", lengths)
# print("target_variable:", target_variable) # shape (max_target_len, batch_size)
# print("mask:", mask)
# print("max_target_len:", max_target_len)

### Defining model architecture

**Computation Process:**
    
    1) Convert word indexes to embeddings.
    2) Pack padded batch of sequences for RNN module.
    3) Forward pass through LSTM.
    4) Unpack padding.
    5) Sum bidirectional LSTM outputs.
    6) Return output and final hidden state.


#### Encoder

In [10]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers,
                            dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq) # output shape (L, N, embedding_dim); output shape (*, embedding_dim) where * is the shape of the input_seq
        # Pack padded batch of sequences for RNN module
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through LSTM
        outputs, hidden = self.lstm(packed, hidden) # input shape (L, N, hidden_size) or packed sequence object
        # Unpack padding
        outputs, _ = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional LSTM outputs.
        # Note that here the outputs from bigru will have the shape = (max_length, batch_size, hidden_size x num_directions)
        # And we want to sum the output from different directions at the same time step.
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden # output shape is (max_length, batch_size, hidden_size) where hidden_size is sum of the bidirection outputs

#### Attention layer

In [11]:
# Luong attention layer
class Attn(torch.nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = torch.nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = torch.nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = torch.nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        # hidden.expand(encoder_output.size(0), -1, -1): hidden (1, batch_size, hidden_size) -> (max_len, batch_size, hidden_size)
        # after cat: (max_len, batch_size, 2 * hidden_size)
        # after self.attn: (max_len, batch_size, hidden_size)
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        # attn_energies has shape = (max_length, batch_size)
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t() # attn_energies shape is (max_length, batch_size), after transpose is (batch_size, max_len)

        # Return the softmax normalized probability scores (with added dimension) # (zhiwei's comment: softmax across dimension 1 means finding the attention weights for each word (dimension 1 is the max_length i.e. sequence length)
        return F.softmax(attn_energies, dim=1).unsqueeze(1) # (batch_size,1,max_length)

#### Decoder

**Computation Process:**

    1) Get embedding of current input word.
    2) Forward through unidirectional LSTM.
    3) Calculate attention weights from the current LSTM output from (2).
    4) Multiply attention weights to encoder outputs to get new "weighted sum" context vector.
    5) Concatenate weighted context vector and LSTM output using Luong
    6) Predict next word using Luong eq. 6 (without softmax).
    7) Return output and final hidden state.

In [12]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_seq, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time (across batch)
        # Get embedding of current input word
        embedded = self.embedding(input_seq)
        embedded = self.embedding_dropout(embedded)
        rnn_output, hidden = self.lstm(embedded, last_hidden)
        # Calculate attention weights from the current LSTM output
        attn_weights = self.attn(rnn_output, encoder_outputs) # attn_weights shape (batch_size,1,max_length), where max_length is the softmax attn weights for each input token
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # (batch_size, 1, max_length) X (batch_size, max_length, hidden_size) -> (batch_size, 1, hidden_size)
        # Concatenate weighted context vector and LSTM output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0) # (1, batch_size, hidden_size)->(batch_size, hidden_size)
        context = context.squeeze(1) # (batch_size, hidden_size)
        concat_input = torch.cat((rnn_output, context), 1) # (batch_size, 2 * hidden_size)
        concat_output = torch.tanh(self.concat(concat_input)) # (batch_size, 2 * hidden_size) # tanh activation on the concatenated context and rnn output
        # Predict next word using Luong eq. 6
        output = self.out(concat_output) # (batch_size, hidden_size)
        output = F.softmax(output, dim=1) # (batch_size, output_size)
        # Return output and final hidden state
        return output, hidden # output shape (batch_size, output_size), hidden (h_n, c_n) where h_n (D * num_layers, N, h_out), c_n (D * num_layers, N, h_cell)

### Training Functions

In [13]:
def maskNLLLoss(inp, target, mask): # inp is decoder_output which is softmax prob. of each word. shape (batch_size, voc.num_words)
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [14]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    # lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    # decoder_hidden = encoder_hidden[:decoder.n_layers]
    decoder_hidden = (encoder_hidden[0][:decoder.n_layers], encoder_hidden[1][:decoder.n_layers])
    # print('decoder_hidden shape', decoder_hidden.shape)

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            # print('decoder_input.shape', decoder_input.shape)
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1) # (zw's comment) target_variable[t] means length t (next one) since index start from 0
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [15]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch
        mask = mask.bool()

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

### Initialise model

In [16]:
# Configure models
# attn_model = 'dot'
attn_model = 'general'
# attn_model = 'concat'
hidden_size = 1024
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64
model_name = f'lstm-{attn_model}-{encoder_n_layers}-{decoder_n_layers}-{hidden_size}'

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = f"{FOLDER_PATH}/checkpoints/{model_name}/2-2_1024/1000_checkpoint.tar"
# loadFilename = None

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

<ipython-input-16-cac9c4e6c7b6>:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(loadFilename)


Building encoder and decoder ...
Models built and ready to go!


### Run training

In [ ]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 1000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# Run training iterations
print("Starting Training!")
save_dir = f"{FOLDER_PATH}/checkpoints"
trainIters(model_name, voc, qa_pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 501; Percent complete: 50.1%; Average loss: 0.3419
Iteration: 502; Percent complete: 50.2%; Average loss: 0.3286
Iteration: 503; Percent complete: 50.3%; Average loss: 0.3409
Iteration: 504; Percent complete: 50.4%; Average loss: 0.4300
Iteration: 505; Percent complete: 50.5%; Average loss: 0.3816
Iteration: 506; Percent complete: 50.6%; Average loss: 0.4057
Iteration: 507; Percent complete: 50.7%; Average loss: 0.5884
Iteration: 508; Percent complete: 50.8%; Average loss: 0.5654
Iteration: 509; Percent complete: 50.9%; Average loss: 0.3546
Iteration: 510; Percent complete: 51.0%; Average loss: 0.3097
Iteration: 511; Percent complete: 51.1%; Average loss: 0.3886
Iteration: 512; Percent complete: 51.2%; Average loss: 0.3589
Iteration: 513; Percent complete: 51.3%; Average loss: 0.3358
Iteration: 514; Percent complete: 51.4%; Average loss: 0.3815
Iteration: 515; Percent complete: 51.5%; Average loss: 0.596

### Evaluation Functions

In [17]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder,device):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self._device = device

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = (encoder_hidden[0][:decoder.n_layers], encoder_hidden[1][:decoder.n_layers])
        # Initialize decoder input with SOS_token
        decoder_input = torch.LongTensor([[SOS_token]])
        decoder_input = decoder_input.to(device)
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=self._device, dtype=torch.long)
        all_scores = torch.zeros([0], device=self._device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [18]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    # lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

### Manual Evaluation

In [19]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder,device)


# Begin chatting
evaluateInput(encoder, decoder, searcher, voc)

> what is recursion
Bot: recursion is a programming technique where a function calls itself in its own definition it allows for solving complex problems by breaking them down into smaller simpler subproblems that are solved recursively recursion can be used to solve problems that exhibit a divide and conquer or topdown approach where a problem is divided into smaller subproblems until a base case is reached recursion can be powerful but should be used with caution to prevent infinite loops or stack overflow errors
> Explain the concept of recursion to me
Bot: recursion is a programming technique where a function calls itself in its own definition it allows for solving complex problems by breaking them down into smaller simpler subproblems that are solved recursively recursion can be used to solve problems that exhibit a divide and conquer or topdown approach where a problem is divided into smaller subproblems until a base case is reached recursion can be powerful but should be used wit

### BERT Score evaluation

In [ ]:
!pip install bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00


In [ ]:
# check your installation
import bert_score
print(bert_score.__version__)

0.3.12


In [ ]:
# Get n random unique questions from questions
from bert_score import score
questions_evaluation = [q[0] for q in qa_pairs]
answers_evaluation = [q[1] for q in qa_pairs]

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    # lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder,device)

In [ ]:
generated_answers = []
for question in questions_evaluation:
    question = normalizeString(question)
    # Evaluate sentence
    output_words = evaluate(encoder, decoder, searcher, voc, question)
    # Format and print response sentence
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    generated_answers.append(' '.join(output_words))

In [ ]:
# Compute the BERT score for each generated answer and the answer
eval_df = pd.DataFrame({'question': questions_evaluation, 'answer': answers_evaluation, 'generated_answer': generated_answers})
P, R, F1 = score(generated_answers, answers_evaluation, lang='en', verbose=True, rescale_with_baseline=True)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/30 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/59 [00:00<?, ?it/s]

done in 5.97 seconds, 632.73 sentences/sec


In [ ]:
eval_df['P'] = P
eval_df['R'] = R
eval_df['F1'] = F1
# save to csv
save_path = f'{FOLDER_PATH}/evaluation/{model_name}.csv'
eval_df.to_csv(save_path, index=False)
print(f"saved to {save_path}")
eval_df.head()

saved to /content/drive/My Drive/Colab Notebooks/nlc/project/evaluation/lstm-general-2-2-1024.csv


,question,answer,generated_answer,P,R,F1
0,how does randomised algorithm work,the algorithm typically uses uniformly random ...,the algorithm typically uses uniformly random ...,1.000000,1.000000,1.000000
1,what do you mean by bestfirst search,bestfirst search is a search algorithm which e...,bestfirst search is a search algorithm which e...,1.000000,1.000000,1.000000
2,how do you explain a daemon,daemon disk and execution monitor is a process...,daemon is a program that runs in the backgroun...,1.000000,1.000000,1.000000
3,what is phonetic algorithm,a phonetic algorithm is an algorithm for index...,a phonetic algorithm is an algorithm for index...,0.999999,0.999999,0.999999
4,what do you mean by uniform costsearch,a tree search that finds the lowestcost route ...,a tree search that finds the lowestcost route ...,1.000000,1.000000,1.000000


In [ ]:
# average BERT Score
print(f'Avg. P: {P.mean():.4f}, Avg. R: {R.mean():.4f}, Avg. F1: {F1.mean():.4f}')
# minimum BERT Score
print(f'Min. P: {P.min():.4f}, Min. R: {R.min():.4f}, Min. F1: {F1.min():.4f}')

Avg. P: 0.9521, Avg. R: 0.9543, Avg. F1: 0.9531
Min. P: -0.6662, Min. R: -0.1103, Min. F1: -0.2307


In [ ]:
# Display the questions and generated answers who have negative f1
poor_eval_df = eval_df[eval_df['F1'] < 0]
# sort by f1 in descending
poor_eval_df = poor_eval_df.sort_values(by='F1', ascending=True)
poor_eval_df

,question,answer,generated_answer,P,R,F1
3070,explain in brief about data imputation,data imputation is the process of filling in m...,data imputation is the process of filling in m...,-0.666181,0.298758,-0.230679
1691,what are the disadvantages array implementatio...,the no of nodes needed cant be predicted when ...,the no of nodes needed cant be declared a list...,-0.311747,-0.070371,-0.192183
2636,what are the states of a process,new runnin waitingready and terminated,new runnin waitingready and terminated,-0.258682,-0.100382,-0.178927
57,how do you explain topological sort,finds linear order of nodes eg jobs based on t...,finds linear order of nodes eg jobs based on t...,-0.193739,0.029004,-0.083156
1008,what is topological sort,finds linear order of nodes eg jobs based on t...,finds linear order of nodes eg jobs based on t...,-0.193739,0.029004,-0.083156
2221,define topological sort,finds linear order of nodes eg jobs based on t...,finds linear order of nodes eg jobs based on t...,-0.193739,0.029004,-0.083156
399,what do you mean by topological sort,finds linear order of nodes eg jobs based on t...,finds linear order of nodes eg jobs based on t...,-0.193739,0.029004,-0.083156
224,what do you mean by a priority queue,the priority queue is a data structure in whic...,the priority queue is a data structure in whic...,-0.132608,0.002581,-0.064217
1198,how do you explain a priority queue,the priority queue is a data structure in whic...,the priority queue is a data structure in whic...,-0.117299,0.040921,-0.037775
1493,what are the different types of scheduling alg...,the scheduling algorithms decide which process...,the scheduling algorithms lifo insertion under...,-0.089561,0.064208,-0.012226


#### Visualisation
Visualise the distribution of positive and negative BERT score answers

In [ ]:
!pip install plotly-express

In [ ]:
import plotly.express as px

In [ ]:
# plot histogram of the f1 using plotly express
fig = px.histogram(eval_df[eval_df['F1'] < 0], x='F1', nbins=20, title='Histogram Plot of BERT (F1) Score of Generated Answer vs Answer')
fig.update_layout(xaxis_title='F1 Score', yaxis_title='Count')
fig.show()

### Hyperparameter Evaluation

Compare the average BERT score across the different hyperparameters

In [ ]:
# Compare the BERT score
# load the evaluation csv files
import pandas as pd
evaluation_csv_files = ['lstm-dot-2-2-1024.csv', 'lstm-dot-4-4-1024.csv', 'lstm-general-2-2-1024.csv', 'lstm-general-4-4-1024.csv']
highest_f1 = 0
for eval_file in evaluation_csv_files:
    eval_df = pd.read_csv(f'{FOLDER_PATH}/evaluation/{eval_file}')
    print(f'file: {eval_file}\t Avg. P: {eval_df["P"].mean():.4f}, Avg. R: {eval_df["R"].mean():.4f}, Avg. F1: {eval_df["F1"].mean():.4f}')
    if eval_df['F1'].mean() > highest_f1:
        highest_f1 = eval_df['F1'].mean()
        highest_f1_file = eval_file
print(f'{highest_f1_file} has the highest avg bert score: {highest_f1}')

file: lstm-dot-2-2-1024.csv	 Avg. P: 0.9494, Avg. R: 0.9537, Avg. F1: 0.9515
file: lstm-dot-4-4-1024.csv	 Avg. P: 0.7872, Avg. R: 0.7901, Avg. F1: 0.7884
file: lstm-general-2-2-1024.csv	 Avg. P: 0.9521, Avg. R: 0.9543, Avg. F1: 0.9531
file: lstm-general-4-4-1024.csv	 Avg. P: 0.8036, Avg. R: 0.8068, Avg. F1: 0.8049
lstm-general-2-2-1024.csv has the highest avg bert score: 0.9531026439086092


LSTM with 2 encoder and 2 decoder layers, 1024 hidden size using dot attention gives the highest BERT Score

### Telegram Implementation

In [ ]:
!pip install python-telegram-bot --upgrade
!pip install nest_asyncio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 654.9/654.9 kB 13.2 MB/s eta 0:00:00


In [ ]:
import logging
import asyncio
import nest_asyncio
from telegram import Update
from telegram.ext import Application, MessageHandler, filters, ContextTypes

# Apply the nest_asyncio patch for Jupyter environments
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Define the function to process user input and generate responses using your model
async def respond(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_message = update.message.text
    # Preprocess user message if necessary
    input_sentence = user_message  # Or use preprocess_sentence(user_message)

    # Generate a response using your evaluate function
    response = evaluate(encoder, decoder, searcher, input_sentence)

    # Send the response back to the user
    await update.message.reply_text(response)

# Main function to set up and run the bot
async def main():
    # Replace 'YOUR_TOKEN' with your bot's API token
    application = Application.builder().token('Replace token here').build()

    # Add handler for text messages
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, respond))

    # Initialize, start, and poll the bot
    await application.initialize()
    await application.start()
    await application.updater.start_polling()
    # Keep the bot running
    await asyncio.Event().wait()

# Run the bot using asyncio
asyncio.run(main())

KeyboardInterrupt: 